In [ ]:
# 데이터셋 명목형 / 숫자형 나누어서 계산해보기

In [ ]:
# 요일을 one_hot_encoding으로
    # functional 왼쪽 오른쪽으로 나누어서 진행
    # val_loss < loss 보다 작으므로 과적함 => 그러려면 층을 줄여주고 노드수도 줄여주는게 좋다
    
    # 지수러닝으로 줄여줬다 loss < val_loss 보다 (점심 , 저녁 둘다)


# Next
# 음식들도 가중치가 있는 값으로 변경후에 진행해보기
    # 시작

In [ ]:
# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os
import pandas as pd

# 새 섹션

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_ln = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/식수인원_MLP_DNN/train_one_hot.csv')

test_ln = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/식수인원_MLP_DNN/test_one_hot.csv')

In [ ]:
train_ln.columns

Index(['numbers', 'dayoff', 'work', 'outsidework', 'workfhome', 'lunch_t',
       'dinner_t', 'Month', 'Date', 'bob', 'soup', 'main', 'bobd', 'soupd',
       'maind', 'present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화'],
      dtype='object')

In [ ]:
test_lunch = test_ln[['numbers', 'dayoff', 'work', 'outsidework', 'workfhome',
       'Month', 'Date', 'bob', 'soup', 'main','present','day_금', 'day_목', 'day_수', 'day_월', 'day_화' ]]

In [ ]:
test_dinner = test_ln[['numbers', 'dayoff', 'work', 'outsidework', 'workfhome',
       'Month', 'Date', 'bobd', 'soupd', 'maind','present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화']]

In [ ]:
# train_ln = [['day','bob','soup','main','present','lunch_t']]
# X_test = [['day','bob','soup','main','present']]

In [ ]:
X = train_ln[['numbers', 'dayoff', 'work', 'outsidework', 'workfhome',
       'Month', 'Date', 'bob', 'soup', 'main','present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화']]
y = train_ln['lunch_t']

In [ ]:

from sklearn.model_selection import train_test_split


X_train, X_vali, y_train, y_vali = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 

                                                    shuffle=True, 

                                                    random_state=1004)


In [ ]:
def model_dnn(Nin, Nh, Nh1, Nout):
    x = keras.layers.Input(shape=(Nin,)) 

    b = keras.layers.BatchNormalization()(x)
    h1 = keras.layers.Activation('elu')(keras.layers.Dense(Nh, kernel_initializer = 'he_normal')(b))
    h1 = keras.layers.Activation('elu')(keras.layers.Dense(Nh, kernel_initializer = 'he_normal')(b))
    h1 = keras.layers.Activation('elu')(keras.layers.Dense(Nh, kernel_initializer = 'he_normal')(b))
    h = keras.layers.Activation('elu')(keras.layers.Dense(Nh, kernel_initializer = 'he_normal')(h1))
    b1 = keras.layers.BatchNormalization()(h)


    b2 = keras.layers.BatchNormalization()(x)
    h2 = keras.layers.Activation('elu')(keras.layers.Dense(Nh1, kernel_initializer = 'he_normal')(b2))
    b2 = keras.layers.Activation('elu')(keras.layers.Dense(Nh1, kernel_initializer = 'he_normal')(h2))
    b2 = keras.layers.Dropout(rate=0.3)(b2)
    

    merge = keras.layers.concatenate([b1, b2])
    merge = keras.layers.Dropout(rate=0.2)(merge)
    merge = keras.layers.BatchNormalization()(merge)


    y = keras.layers.Dense(Nout)(merge)
    model = keras.models.Model(x, y) 
    model.compile(loss='mae', optimizer='nadam') 
    
    return model 


In [ ]:
model = model_dnn(X_train.shape[1], 50, 100, 1)

In [ ]:
model.summary()

Model: "model_24"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 16)]         0           []                               
                                                                                                  
 batch_normalization_105 (Batch  (None, 16)          64          ['input_27[0][0]']               
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_107 (Batch  (None, 16)          64          ['input_27[0][0]']               
 Normalization)                                                                                   
                                                                                           

In [ ]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.0001, s=20)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

history = model.fit(X_train, y_train,
                    validation_data=(X_vali, y_vali),
                    epochs=1000, verbose=1, batch_size=32, callbacks = [early_stopping_cb, lr_scheduler] )

Epoch 1/1000
31/31 [==============================] - 0s 5ms/step - loss: 47.9058 - val_loss: 52.4120 - lr: 1.0000e-04
Epoch 2/1000
31/31 [==============================] - 0s 4ms/step - loss: 50.8922 - val_loss: 52.5134 - lr: 8.9125e-05
Epoch 3/1000
31/31 [==============================] - 0s 4ms/step - loss: 48.8354 - val_loss: 52.4651 - lr: 7.9433e-05
Epoch 4/1000
31/31 [==============================] - 0s 4ms/step - loss: 48.2782 - val_loss: 52.3057 - lr: 7.0795e-05
Epoch 5/1000
31/31 [==============================] - 0s 4ms/step - loss: 47.6567 - val_loss: 52.2943 - lr: 6.3096e-05
Epoch 6/1000
31/31 [==============================] - 0s 4ms/step - loss: 50.7510 - val_loss: 52.2487 - lr: 5.6234e-05
Epoch 7/1000
31/31 [==============================] - 0s 4ms/step - loss: 47.4013 - val_loss: 52.3559 - lr: 5.0119e-05
Epoch 8/1000
31/31 [==============================] - 0s 5ms/step - loss: 47.6664 - val_loss: 52.3723 - lr: 4.4668e-05
Epoch 9/1000
31/31 [============================

In [ ]:
# loss > val_loss => 과대적합이 아닌가..?

In [ ]:
# 예측

lunch = model.predict(test_lunch)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/식수인원_MLP_DNN/sample_submission.csv')
submit['중식계'] = lunch

In [ ]:
# 저녁
X = train_ln[[ 'numbers', 'dayoff', 'work', 'outsidework', 'workfhome',
       'Month', 'Date', 'bobd', 'soupd', 'maind','present', 'day_금', 'day_목', 'day_수', 'day_월', 'day_화']]
y = train_ln['dinner_t']

from sklearn.model_selection import train_test_split


X_train, X_vali, y_train, y_vali = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 

                                                    shuffle=True, 

                                                    random_state=1004)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

history = model.fit(X_train, y_train,
                    validation_data=(X_vali, y_vali),
                    epochs=1000, verbose=1, batch_size=32, callbacks = [early_stopping_cb, lr_scheduler] )

dinner = model.predict(test_dinner)

Epoch 1/1000
31/31 [==============================] - 0s 8ms/step - loss: 47.5575 - val_loss: 52.5636 - lr: 1.0000e-04
Epoch 2/1000
31/31 [==============================] - 0s 6ms/step - loss: 49.0481 - val_loss: 52.6040 - lr: 8.9125e-05
Epoch 3/1000
31/31 [==============================] - 0s 8ms/step - loss: 47.6325 - val_loss: 52.4458 - lr: 7.9433e-05
Epoch 4/1000
31/31 [==============================] - 0s 7ms/step - loss: 47.7413 - val_loss: 52.4277 - lr: 7.0795e-05
Epoch 5/1000
31/31 [==============================] - 0s 6ms/step - loss: 49.5310 - val_loss: 52.5565 - lr: 6.3096e-05
Epoch 6/1000
31/31 [==============================] - 0s 6ms/step - loss: 47.7375 - val_loss: 52.5253 - lr: 5.6234e-05
Epoch 7/1000
31/31 [==============================] - 0s 6ms/step - loss: 48.8860 - val_loss: 52.5152 - lr: 5.0119e-05
Epoch 8/1000
31/31 [==============================] - 0s 11ms/step - loss: 48.2025 - val_loss: 52.4601 - lr: 4.4668e-05
Epoch 9/1000
31/31 [===========================

In [ ]:
submit['석식계'] = dinner

In [ ]:
submit.to_csv('Dnn_functional_05_one_hot_day.csv', index=False)

# 요일을 원핫인코딩으로 준뒤에 학습
    - functional 방식으로 오른쪽 왼쪽을 다르게 주고 학습을 진행했다
    - 점신에서는 loss > loss_val 여서 과적합이 라고 생각
    - 하지만 저녁에서는 loss < loss_val 작으므로 이거는 잘나왔다생각

### 결과
- train : 352.813621
- test : 358.2780774286

